In [1]:
%load_ext watermark
%watermark

2017-10-23T18:30:22+02:00

CPython 3.6.1
IPython 6.2.1

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.13.0-26-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12, 12]
np.random.seed(42)

# KNN - K vecinos más próximos

Vamos a ver como vamos a usar el algoritmo KNN en scikit-learn.

El algoritmo KNN se puede usar tanto en problemas de clasificación (con el estimador [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)) como en problemas de regresión (con el estimador [KNeighborsRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor))

### Cargamos los datos

Para este ejemplo vamos a usar el dataset [CSM (Conventional and Social Media Movies)](https://archive.ics.uci.edu/ml/datasets/CSM+%28Conventional+and+Social+Media+Movies%29+Dataset+2014+and+2015) que contiene información de la popularidad en redes sociales de distintas películas así como las ventas en taquilla.

In [4]:
pelis = pd.read_csv("data/datos_peliculas.csv")
pelis.shape

(231, 12)

In [5]:
pelis.head()

,pelicula,año,ratings,genero,ventas,presupuesto,secuela,vistas_youtube,positivos_youtube,negativos_youtube,comentarios,seguidores_agregados
0,13 Sins,2014,6.3,8,9130,4000000.0,1,3280543,4632,425,636,1120000.0
1,22 Jump Street,2014,7.1,1,192000000,50000000.0,2,583289,3465,61,186,12350000.0
2,3 Days to Kill,2014,6.2,1,30700000,28000000.0,1,304861,328,34,47,483000.0
3,300: Rise of an Empire,2014,6.3,1,106000000,110000000.0,2,452917,2429,132,590,568000.0
4,A Haunted House 2,2014,4.7,8,17300000,3500000.0,2,3145573,12163,610,1082,1923800.0


Vamos a eliminar el título de las películas

In [6]:
pelis = pelis.drop("pelicula", axis=1)

Vamos a usar este dataset para probar KNN en clasificación y en regresión

In [7]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split

### KNN para problemas de clasificación

Probamos KNN para clasificación, en concreto vamos a suponer que queremos predecir el género de una película en función de su popularidad.

In [8]:
from sklearn.metrics import f1_score

variable_objetivo_clasificacion = "genero"
variables_independientes_clasificacion = pelis.drop(
    variable_objetivo_clasificacion, axis=1).columns

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_clasificacion],
    pelis[variable_objetivo_clasificacion], test_size=0.20)

In [10]:
KNeighborsClassifier?

Los parámetros más importantes a la hora de usar `KNeighborsClasifier` son:

- **n_neighbors**: El valor de K, es decir el número de vecinos que considerar a la hora de asignar una clase.
- **weights**: A la hora de votar, que importancia dar a los vecinos. Si elegimos `auto` asigna la misma importancia a todos los vecinos. Si elegimos `distance` asigna importancia a los vecinos en función de la distancia de los vecinos al punto a clasificar
- **metric**: La métrica a la hora de medir la distancia entre los puntos. Si se usa distancia de Minkowsky se puede elegir p con el parámetro `p`, que por defecto es 2 (lo que computa la distancia euclidiana).

En este caso en particular sabemos que valor elegir de K, ya que podemos asumir que el número de categorías del dataset es el total de categorías de películas del dataset de entrenamiento.

In [11]:
k_categorias = len(y_train.unique())
k_categorias

10

In [18]:
clasificador_knn = KNeighborsClassifier(n_neighbors=10, 
                                        weights="uniform")

clasificador_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [19]:
preds = clasificador_knn.predict(X_test)
f1_score(y_test, preds, average="micro")

0.3617021276595745

Si ahora entrenamos el estimador con el argumento de pesos `weights="distance"`, vemos que funciona de forma ligeramente mejor.

In [20]:
clasificador_knn = KNeighborsClassifier(n_neighbors=10, 
                                        weights="distance")

clasificador_knn.fit(X_train, y_train)

preds = clasificador_knn.predict(X_test)
f1_score(y_test, preds, average="micro")

0.38297872340425526

Podemos usar el método `kneighbors` para devolver los k vecinos de un punto en concreto

In [21]:
X_test.iloc[0]

año                         2015.0
ratings                        7.7
ventas                  49500000.0
presupuesto             30000000.0
secuela                        1.0
vistas_youtube          11476882.0
positivos_youtube          40496.0
negativos_youtube           1383.0
comentarios                 4435.0
seguidores_agregados           0.0
Name: 218, dtype: float64

In [22]:
distancia, indice = clasificador_knn.kneighbors(
    [X_test.iloc[0]], n_neighbors=1)
distancia, indice

(array([[8648469.61304606]]), array([[156]], dtype=int64))

In [23]:
X_train.iloc[indice[0]]

,año,ratings,ventas,presupuesto,secuela,vistas_youtube,positivos_youtube,negativos_youtube,comentarios,seguidores_agregados
191,2015,7.3,42500000,25000000.0,1,11036701,50002,1005,3525,776000.0


Vemos que el vecino más cercano es similar.

### KNN para problemas de regresión

Vamos a utilizar ahora el algoritmo KNN para un problema de regresión, KNN funciona igual para hacer regresiones, simplemente que en vez de una votación donde la clase más común entre los vecinos más próximos es la elegida, se hace una interpolación de los valores de la variable numérica objetivo de los vecinos.

en concreto vamos a estimar las ventas de entradas en taquilla de una película en función de su popularidad online y presupuesto.

In [24]:
from sklearn.metrics import mean_squared_error

variable_objetivo_regresion = "ventas"
variables_independientes_regresion = pelis.drop(
    variable_objetivo_regresion, axis=1).columns

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_regresion],
    pelis[variable_objetivo_regresion], test_size=0.20)

Usamos la implementación en sklearn `KNeighborsRegressor` para problemas de regresión. Tiene los mismos hiperparámetros que `NeighborsClassifier`.

In [26]:
KNeighborsRegressor?

In [27]:
regresor_knn = KNeighborsRegressor(n_neighbors=10, weights="distance")

regresor_knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='distance')

In [28]:
preds = regresor_knn.predict(X_test)
preds

array([5.77598582e+06, 1.99218779e+08, 5.22718267e+07, 3.04748097e+06,
       7.80062231e+07, 4.78727214e+07, 5.21240238e+07, 5.93022167e+07,
       7.49468197e+07, 9.12329679e+07, 7.03489473e+07, 1.82692317e+08,
       2.82445048e+07, 1.08624687e+07, 3.97173720e+07, 4.03493052e+07,
       3.63396779e+07, 3.23651506e+07, 1.21506362e+07, 2.18446367e+07,
       1.29536895e+08, 1.12369979e+07, 4.43514409e+07, 4.01796045e+07,
       1.32035921e+07, 1.02480559e+08, 9.88894202e+07, 9.56558550e+07,
       7.74208639e+07, 3.06222002e+08, 9.01985135e+07, 3.57704603e+06,
       7.15873565e+06, 1.59847370e+08, 5.88276776e+07, 2.23216488e+08,
       1.27252372e+07, 3.99596725e+07, 3.71233940e+07, 8.00191716e+07,
       3.10354554e+07, 3.86931786e+07, 1.94912972e+08, 2.23710764e+08,
       5.81636294e+07, 2.26283159e+07, 1.19539899e+07])

In [29]:
np.sqrt(np.abs(mean_squared_error(y_test, preds)))

43959789.048082694

Ahora vemos el funcionamiento en validación cruzada tanto del clasificador como el regresor

In [30]:
from sklearn.model_selection import cross_val_score

In [31]:
error_validacion_cruzada_clasificacion = np.sqrt(np.abs(
    cross_val_score(KNeighborsClassifier(n_neighbors=k_categorias,
                                         weights="distance"), 
                X=pelis[variables_independientes_clasificacion],
               y=pelis[variable_objetivo_clasificacion], 
               scoring="f1_micro"
        ).mean()
      )
)
print("La puntuación F1 de KNN para clasificacion en este dataset es {:.2f}".format(
    error_validacion_cruzada_clasificacion))

La puntuación F1 de KNN para clasificacion en este dataset es 0.54


C:\Users\manuel\Anaconda3\envs\data\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


In [32]:
error_validacion_cruzada_regresion = np.sqrt(np.abs(
    cross_val_score(KNeighborsRegressor(weights="distance"), 
                X=pelis[variables_independientes_regresion],
               y=pelis[variable_objetivo_regresion], 
               scoring="neg_mean_squared_error"
        ).mean()
      )
)
print("El error RMSE de KNN para regresión en este dataset es {:.2f}$".format(
    error_validacion_cruzada_regresion))

El error RMSE de KNN para regresión en este dataset es 69488698.28$
